# Sentiment-Analyse für Kundenrezensionen

Wir trainieren ein Neuronales Netzwerk darin zu erkennen, ob eine Rezension eines Kunden eher positiv oder eher negativ ausfällt.

### Daten
Der Datensatz umfasst Rezensionen von Nutzern für die DKB und für drei andere Banken, die in Webforen wie Trustpilot veröffentlicht wurden. Zusätzlich zum Text bewertet der Kunde die Bank mit 1 bis 5 Sternen. Diese Sterne werden als Label verwendet zum Trainieren des Netzwerks. Eine Rezension mit 5 Sternen gilt als positiv, eine mit einem Stern als negativ und alles dazwischen als neutral (wie wir sehen werden, vergeben die meisten Kunden entweder 1 oder 5 Sterne, sodass die Daten so einigermaßen gleichmäßig verteilt werden).

### Vorverarbeitung
Dem neuronalen Netzwerk wird ein Vorverständnis von deutscher Sprache mitgegeben. Dazu werden die Wörter aus den Rezensionen semantisch vorverarbeitet. Hierzu kommt die Bibliothek Fasttext von Facebook zum Einsatz, deren Word Embeddings benutzt werden, um die Wörter in 300-dimensionale numerische Vektoren zu transformieren. Die Dimensionen geben verschiedene Aspekte von Bedeutung und Inhalt wieder. Wörter mit ähnlicher Bedeutung liegen dabei nah beieinander. Dies wird weiter unten im Notebook an einem Beispiel gezeigt.

Die Nutzung von Fasttext ist ein Beispiel für Transfer Learning.

### Architektur des neuronalen Netzwerks
Es wird ein rekurrentes neuronales Netzwerk trainiert, das Sequenzen von Wörter (den Text der Rezension) verarbeitet. In dem neuronalen Netzwerk kommen einige zusätzliche Technologien zum Einsatz wie LTSM (Long Short-Term Memory), die sich für die Verarbeitung von natürlicher Sprache bewährt haben. Diese zu erläutern, würde hier den Rahmen sprengen.

## Vorbereitung

Zuerst importieren wir einige Bibliotheken und Hilfsfunktionen

In [ ]:
import utils
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import matplotlib.pyplot as plt

Dann lesen wir die Rezensionen für DKB und anderen Banken eingeladen und die ersten Einträge angezeigt

In [ ]:
reviews_df = pd.read_csv('Rezensionen 20190828.csv')
reviews_df.head()

In [ ]:
reviews_df.source.value_counts()

**Beobachtung:** Die Rezensionen stammen aus unterschiedlichen Quellen

In [ ]:
reviews_df.code.value_counts()

**Beobachtung:** Neben der DKB gibt es auch Rezensionen für N26, ING und Comdirect

In [ ]:
reviews_df.hist(column='stars', by='code', figsize=(10, 6))
plt.show()

Die obigen Histogramme für jede der vier Banken, wieviele Reviews mit 1, 2, 3, 4 und 5 Sternen abgegeben wurden.

**Beobachtung:** Nur wenige Kunden vergeben 2 bis 4 Sterne, die meisten vergeben entweder die Minimal- oder die Maximalwertung

**Beobachtung:** ING schneidet sehr gut ab, mit vielen 5-Sterne Bewertungen, bei DKB herrscht eine negative Meinung vor

# Sentiment-Analyse

#### Normalisierung

Ehe wir die Texte mit einem Neuronalen Netzwerk verarbeiten, säubern wir die Daten. Dazu gehört es, Sonderzeichen und Zahlen zu entfernen, die Leerzeichen zu normalisieren etc.

In [ ]:
import re

reviews = [text.replace('\xa0', ' ') for text in reviews_df.text]    # fix whitespace
reviews = [re.sub('[.,;:()\'?"!\\-]', ' ', text) for text in reviews]    # replace punctuation
reviews = [re.sub('[0-9]+', 'NUMBER', text) for text in reviews]    # replace numbers

Ein Beispiel nach der Vorverarbeitung:

In [ ]:
print(reviews[0])

Für die Verwendung der Texte in einem KI-Modell müssen wir diese numerisch kodieren. Eine Möglichkeit dafür ist TFIDF. Dabei werden Wörter entsprechend ihrer Anzahl in einem Text relativ zu ihrer Anzahl in allen Texten gewichtet. Diese Gewichtung ist dann die numerische Codierung die zum Trainieren eines KI-Modells verwendet werden kann.

Wörter die in über 90% der Texte vorkommen sind nicht aussagekräftig für den Text und werden von uns daher ausgeschlossen. Wir schließen ebenfalls Wörter aus die in weniger als 10 Texten vorkommen, aus diesen kann ein KI-Modell ebenfalls nichts lernen. Von den übrigen Wörtern nehmen wir die 1000 häufigsten.

In [ ]:
import sklearn.feature_extraction.text

tfidf_encoder = sklearn.feature_extraction.text.TfidfVectorizer(
    ngram_range=(1, 1),
    min_df=10,
    max_df=0.9,
    strip_accents='unicode',
    use_idf=1,
    smooth_idf=1,
    sublinear_tf=1,
    norm='l2',
    max_features=1000,
)

tfidf_encoder.fit(reviews)

Wir können uns jetzt anschauen wieviele Wörter ausgeschlossen wurden ...

In [ ]:
len(tfidf_encoder.stop_words_)

... und welche das beispielhaft sind.

In [ ]:
list(tfidf_encoder.stop_words_)[:10]

Wir schauen uns nun die 1000 Wörter an die verwendet werden.

In [ ]:
' '.join(sorted(tfidf_encoder.vocabulary_.keys()))

Wir wenden nun die numerische Codierung an.

In [ ]:
reviews_tfidf = tfidf_encoder.transform(reviews).toarray()

Die ersten hundert Einträge in einem codierter Text sehen z.B. wie folgt aus.

Dabei bekommt jedes der 1000 Wörter, das im Text nicht auftaucht eine 0. Die vorhandenen Wörter bekommen eine Zahl entsprechend der durch TFIDF festgelegten Gewichtung.

In [ ]:
reviews_tfidf[0][:100]

### Datenaufbereitung

Nun vergeben wir die Labels. Wie eingangs beschrieben, verwenden wir hierzu die folgende Zuordnung:

In [ ]:
stars_mapping = {1: 0, 2: 1, 3: 1, 4: 1, 5: 2}

In [ ]:
targets = reviews_df.stars.map(stars_mapping)
targets.head()

Nun ordnen wir die semantischen Wortvektoren, die wir bereits oben kennengelernt haben, den Wörter aus den Rezensionen zu. Wir beschränken uns dabei jeweils auf die ersten 200 Wörter einer Rezension.

### Implementation des Neuronalen Netzwerks

Zuerst laden wir die nötigen Bibliotheken, um ein Neuronales Netzwerk in Python aufzusetzen

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras import optimizers

Der folgende Code baut die Schichten des Neuronalen Netzwerks auf

In [ ]:
model = Sequential()

#input layer
model.add(BatchNormalization(input_shape=(1000,)))

#hidden layers
model.add(Dense(100, activation="relu"))
model.add(Dense(10, activation="relu"))
model.add(Dropout(rate=0.3))

#output layer
model.add(Dense(3, activation="softmax"))

model.compile(optimizer=optimizers.Adam(lr=0.001,decay=0.0001),loss="categorical_crossentropy",metrics=['acc'])

Mit der Funktion *model.summary()* überprüfen wir die Struktur des neuronalen Netzes

In [ ]:
model.summary()

### Training

Wir teilen nun die Daten wie immer auf in Trainingsdaten und Testdaten

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(reviews_tfidf, to_categorical(targets), test_size=0.2, random_state=71)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

Jetzt trainieren wir das Modell

In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=3, validation_data=[X_test, y_test])

#### Auswertung

Wie üblich schauen wir uns die Ergebnisse an und plotten die Genauigkeiten (d.h. der Anteil derjenigen Rezensionen, bei denen die korrekte Kategorie vorhergesagt wurde) auf den Trainingsdaten und den Testdaten über die 5 Epochen

In [ ]:
plt.plot(model.history.history['acc'])
plt.plot(model.history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

Die Genauigkeit auf den Testdaten liegt bei über 80%. Die genaue Zahl berechnen wir von Hand

In [ ]:
# Wir führen das Modell auf den Testdaten aus
scores_test = model.predict(X_test)
pred_test = scores_test.argmax(axis=1)
true_test = y_test.argmax(axis=1)

# Wir berechnen die Anzahl der korrekten Vorhersagen und deren Prozentsatz
correct = sum(pred_test == true_test)
print('Test Accuracy', correct / len(pred_test))

Wir erhalten eine Genauigkeit von ca. 83%, was sich durchaus sehen lassen kann!